In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/fakeNews.csv', on_bad_lines='skip', engine='python')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [8]:
#Drop Nan Values
df=df.dropna()


In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
y.value_counts()

,count
label,
0.0,6976
1.0,5307


In [12]:
X.shape

(12283, 4)

In [13]:
y.shape

(12283,)

In [14]:
import tensorflow as tf

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [16]:
### Vocabulary size
voc_size=10000

### Onehot Representation

In [17]:
messages=X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages.reset_index(inplace=True)

In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [24]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [31]:
# Words Left After Stemming
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [26]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[7295, 4744, 7262, 9470, 4974, 4365, 4164, 2806, 516, 8655],
 [3778, 2119, 1614, 5059, 7488, 6770, 6516],
 [7048, 4396, 6071, 549],
 [2703, 6651, 6204, 6235, 1321, 9649],
 [7841, 7488, 2737, 3819, 6969, 6152, 7488, 4695, 5436, 7338],
 [9444,
  6253,
  150,
  8046,
  9136,
  8826,
  7926,
  3768,
  6824,
  6716,
  9786,
  8895,
  1271,
  234,
  6516],
 [3310, 8037, 5474, 1163, 2570, 8080, 8270, 7515, 1947, 7134, 6216],
 [7463, 3412, 5740, 9471, 8957, 952, 8826, 3796, 1947, 7134, 6216],
 [2243, 9468, 410, 6793, 6990, 1489, 532, 8949, 8826, 7546],
 [7731, 6233, 9831, 5534, 7299, 7658, 4048, 1274],
 [5454, 8395, 1302, 7554, 9569, 5636, 9081, 4541, 2548, 6464, 8363],
 [6235, 2252, 4974, 1489, 8826, 8957],
 [9691, 9556, 3687, 3901, 1401, 7457, 5304, 8678, 4121],
 [5542, 8851, 8493, 8954, 4960, 6131, 6724, 1947, 7134, 6216],
 [962, 8166, 4248, 9992, 7962, 1947, 7134, 6216],
 [6969, 7843, 3677, 6174, 1497, 897, 5442, 9644, 3770, 9984],
 [8817, 8018, 2119],
 [3063, 6653, 8801, 659, 8826, 4053,

### Embedding Representation

In [27]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2806  516 8655]
 [   0    0    0 ... 7488 6770 6516]
 [   0    0    0 ... 4396 6071  549]
 ...
 [   0    0    0 ... 9845 1513 6516]
 [   0    0    0 ... 1947 7134 6216]
 [   0    0    0 ... 4579  503 6516]]


In [28]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 7295,
       4744, 7262, 9470, 4974, 4365, 4164, 2806,  516, 8655], dtype=int32)

In [41]:
# Model Creation

# Previous Old Syntax -->UserWarning: Argument input_length is deprecated. Just remove it. And a blank summary because the input shape wasn't fully set.
# embeddingVectorFeatures = 40  #Dimensions
# model = Sequential()
# model.add(Embedding(vocabSize, embeddingVectorFeatures, input_length=sentenceLength))
# model.add(LSTM(100))  # 1 LSTM layer with 100 Neurons
# model.add(Dense(1, activation='sigmoid'))  # Dense Layer Because Of Classifications
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())

# Correct Code
embeddingVectorFeatures = 40
model = Sequential()  # Sequential Model
model.add(Embedding(input_dim=voc_size, output_dim=embeddingVectorFeatures, input_shape=(sent_length,)))  # Embedding Layer
model.add(Bidirectional(LSTM(100))) # LSTM Layer
model.add(Dense(1, activation='sigmoid'))  # Dense Layer For Classification
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Model Compilation
model.summary()


# Why This Works
# input_shape=(sentenceLength,) properly initializes the input dimensions.
# input_length= is kept in some old tutorials but no longer needed in newer TensorFlow/Keras versions.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 20, 40)         │       400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 200)            │       112,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 513,001 (1.96 MB)

 Trainable params: 513,001 (1.96 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
from tensorflow.keras.layers import Dropout
embeddingVectorFeatures = 40
model2 = Sequential()
model2.add(Embedding(input_dim=voc_size, output_dim=embeddingVectorFeatures, input_shape=(sent_length,)))
model2.add(Dropout(0.3))
model2.add(Bidirectional(LSTM(100)))
model2.add(Dropout(0.3))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 20, 40)         │       400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 20, 40)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 200)            │       112,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 513,001 (1.96 MB)

 Trainable params: 513,001 (1.96 MB)

 Non-trainable params: 0 (0.00 B)

In [44]:
len(embedded_docs),y.shape

(12283, (12283,))

In [45]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [46]:
X_final.shape,y_final.shape

((12283, 20), (12283,))

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [48]:
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7314 - loss: 0.4788 - val_accuracy: 0.9147 - val_loss: 0.2086
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9433 - loss: 0.1447 - val_accuracy: 0.9179 - val_loss: 0.1977
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9776 - loss: 0.0786 - val_accuracy: 0.9179 - val_loss: 0.2397
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9851 - loss: 0.0492 - val_accuracy: 0.9077 - val_loss: 0.2886
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9917 - loss: 0.0325 - val_accuracy: 0.9144 - val_loss: 0.3152
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9934 - loss: 0.0236 - val_accuracy: 0.9063 - val_loss: 0.3816
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9961 - loss: 0.0157 - val_accuracy: 0.9080 - val_loss: 0.4038
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9977 - loss: 0.0097 - val_accur

### Performance Metrics And Accuracy

In [55]:
# Predictions of Model 1 (Without DropOut)
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)  # Convert probabilities to binary labels


127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [56]:
# Perforamance of Model 1
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.42328564380858413

In [57]:
from sklearn.metrics import confusion_matrix

In [59]:
confusion_matrix(y_test,predictions)

array([[  15, 2291],
       [  47, 1701]])

In [60]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.24      0.01      0.01      2306
         1.0       0.43      0.97      0.59      1748

    accuracy                           0.42      4054
   macro avg       0.33      0.49      0.30      4054
weighted avg       0.32      0.42      0.26      4054



In [61]:
# Performance of Model With Droput Layers
predictions2 = model2.predict(X_test)
predictions2 = (predictions2 > 0.5).astype(int)

127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [65]:
# Perforamance of Model 1
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions2)

0.8973852984706463

In [66]:
# Perforamance of Model 2
from sklearn.metrics  import confusion_matrix
confusion_matrix(y_test, predictions2)

array([[2127,  179],
       [ 237, 1511]])

In [67]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions2))

              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91      2306
         1.0       0.89      0.86      0.88      1748

    accuracy                           0.90      4054
   macro avg       0.90      0.89      0.89      4054
weighted avg       0.90      0.90      0.90      4054

